<a href="https://colab.research.google.com/github/osmarbraz/exemplos_Llama2/blob/main/ExemplosAnalisePeriodos_Llama2_Langchain_HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exemplo de Análise de Períodos compostos usando Llama v2.0 7B 8bit usando Langchain e Transformers by HuggingFace

Exemplo de uso do modelo de linguagem grande Llama v2.0.
- Análise de Períodos (oração principal, oração subordinada, oração coordenada)

**Toda a execução ocorre no Google Colaboratory.**

Pré-requisitos:
- Llama 2 não está acessível abertamente e requer solicitação  de acesso. Faça o cadastro no site do https://huggingface.co/join. Depois do login, gere um token de acesso no link https://huggingface.co/settings/tokens.
- Configurar o notebook para usar GPU- Acesse o menu 'Ambiente de Execução -> Alterar o tipo do ambiente de execução -> Acelerador de hardware -> T4 GPU.
-  Executar o notebook selecional 'Ambiente de Execução -> Executar tudo.


**Referências**
https://medium.com/the-techlife/using-huggingface-openai-and-cohere-models-with-langchain-db57af14ac5b


**Notebook de referência:**

https://github.com/guardiaum/tutorial-sbbd2023/blob/main/Prompt_Engineering.ipynb


**Lista dos modelos:**

https://huggingface.co/models


**Artigos referências:**

https://dev.to/nithinibhandari1999/how-to-run-llama-2-on-your-local-computer-42g1


**Link biblioteca Huggingface:**

https://github.com/huggingface/transformers




# 0 - Preparação do ambiente
Preparação do ambiente para execução do exemplo.

## Data e hora de execução

In [ ]:
# Biblioteca de date
from datetime import datetime

data_e_hora_atuais = datetime.now()
data_e_hora_em_texto = data_e_hora_atuais.strftime('%d/%m/%Y %H:%M:%S')

print(data_e_hora_em_texto)

24/04/2024 18:48:22


## Tratamento de logs

Método para tratamento dos logs.

In [ ]:
# Biblioteca de logging
import logging

# Formatando a mensagem de logging
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO)

## Identificando o ambiente Colab

Cria uma variável para identificar que o notebook está sendo executado no Google Colaboratory.

In [ ]:
# Biblioteca do sistema
import sys

# Retorna true ou false se estiver no Google Colaboratory
IN_COLAB = "google.colab" in sys.modules

## Versão Python

In [ ]:
# Biblioteca do sistema
import sys

print("Versão Python:", sys.version)

Versão Python: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


## GPU

In [ ]:
# Import de biblioteca
import torch

GPU_ENABLE = torch.cuda.is_available()

if GPU_ENABLE:
    print("GPU está disponível.")
else:
    print("GPU não está disponível.")

GPU não está disponível.


## Funções auxiliares

Função auxiliar para formatar o tempo como `hh: mm: ss`

In [ ]:
# Import das bibliotecas.
import time
import datetime

def formataTempo(tempo):
    """
      Pega a tempo em segundos e retorna uma string hh:mm:ss
    """
    # Arredonda para o segundo mais próximo.
    tempo_arredondado = int(round((tempo)))

    # Formata como hh:mm:ss
    return str(datetime.timedelta(seconds=tempo_arredondado))

Imprime linhas menores.

In [ ]:
def print_linhas_menores(texto, tamanho=120):
  for i in range(0, len(texto), tamanho):
    print(texto[i:i+tamanho])

# 1 - Instalação das bibliotecas

Bibioteca LangChain é um framework de código aberto para o desenvolvimento de aplicações usando modelos de linguagem grandes.

In [ ]:
!pip install langchain==0.1.16

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


O bitsandbytes é um wrapper leve em torno de funções personalizadas CUDA, em particular otimizadores de 8 bits, multiplicação de matrizes (LLM.int8()) e funções de quantização. É uma dependência do accelerate.

In [ ]:
!pip install bitsandbytes==0.43.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

Accelerate é uma biblioteca que permite que o mesmo código PyTorch seja executado em qualquer configuração distribuída adicionando apenas quatro linhas de código. Otimiza as operações do PyTorch, especialmente na GPU.

https://pypi.org/project/accelerate/

In [ ]:
!pip install accelerate==0.29.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 2.4 MB/s eta 0:00:00


A Biblioteca A Biblioteca Transformers fornece APIs e ferramentas para baixar e treinar facilmente modelos pré-treinados de última geração para Processamento de linguagem natural, Visão computacional, Áudio, etc.

Fornece uma maneira direta de usar modelos pré-treinados.

In [ ]:
!pip install -U transformers==4.40.0

A Biblioteca huggingface-cli fornece vários comandos para interagir com o Hugging Face Hub a partir da linha de comando. Um desses comandos é o login, que permite aos usuários se autenticarem no Hub usando suas credenciais.

In [ ]:
#!pip install huggingface-hub==0.20.3

## Versão bibliotecas instaladas

In [ ]:
!pip freeze

absl-py==1.4.0
accelerate==0.29.3
aiohttp==3.9.5
aiosignal==1.3.1
alabaster==0.7.16
albumentations==1.3.1
altair==4.2.2
annotated-types==0.6.0
anyio==3.7.1
appdirs==1.4.4
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.5.1
arviz==0.15.1
astropy==5.3.4
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.1.0
attrs==23.2.0
audioread==3.0.1
autograd==1.6.2
Babel==2.14.0
backcall==0.2.0
beautifulsoup4==4.12.3
bidict==0.23.1
bigframes==1.2.0
bitsandbytes==0.43.1
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.3.4
bqplot==0.12.43
branca==0.7.1
build==1.2.1
CacheControl==0.14.0
cachetools==5.3.3
catalogue==2.0.10
certifi==2024.2.2
cffi==1.16.0
chardet==5.2.0
charset-normalizer==3.3.2
chex==0.1.86
click==8.1.7
click-plugins==1.1.1
cligj==0.7.2
cloudpathlib==0.16.0
cloudpickle==2.2.1
cmake==3.27.9
cmdstanpy==1.2.2
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.4
cons==0.4.6
contextlib2==21.6.0
contourpy==1.2.1
cryptography==42.0.

# 2 - Carregando o LLM



## 2.1 - Login no huggingface

- Lhama 2 não está acessível abertamente e requer solicitação  de acesso. Faça o cadastro no site do https://huggingface.co/join. Depois do login, gere um token de acesso no link https://huggingface.co/settings/tokens.

Insira o token quando solicitado e depois digite Y para adicionar as credenciais.

In [ ]:
# !huggingface-cli login

Se o seu notebook não for público e não desejar incluir o token de acesso toda vez que for executar o notebook preencha o método save_token.


Crie a variável 'HF_TOKEN' com o valor do **Access Token do HuggingFace**. Abra o Google Colab e navegue até a nova seção 'Secrets' na barra lateral e adicione a variável.

In [ ]:
from huggingface_hub.hf_api import HfFolder

if IN_COLAB:

    from google.colab import userdata

    # ACESS_TOKEN = "<valor_do_acess_token"
    ACCESS_TOKEN  = userdata.get('HF_TOKEN')

    HfFolder.save_token(ACCESS_TOKEN)

Mostrando o usuário conectado

In [ ]:
# !huggingface-cli whoami

## 2.2 - Nome do modelo de linguagem

Define o nome do modelo a ser carregado
Lista dos modelos:
  - https://huggingface.co/meta-llama/Llama-2-7b-hf
  - https://huggingface.co/meta-llama/Llama-2-7b-chat-hf
  - https://huggingface.co/meta-llama/Llama-2-13b-hf
  - https://huggingface.co/meta-llama/Llama-2-13b-chat-hf
  - https://huggingface.co/meta-llama/Llama-2-70b-hf
  - https://huggingface.co/meta-llama/Llama-2-70b-chat-hf

In [ ]:
#nome_modelo = "meta-llama/Llama-2-7b-hf"
nome_modelo = "meta-llama/Llama-2-7b-chat-hf"

#nome_modelo = "meta-llama/Llama-2-13b-hf"
# nome_modelo = "meta-llama/Llama-2-13b-chat-hf"

# Não roda pois exige GPU A100 e mais espaço em disco
#nome_modelo = "meta-llama/Llama-2-70b-hf"
# nome_modelo = "meta-llama/Llama-2-70b-chat-hf"

## 2.3 - Carrega o tokenizador

Carregando o **tokenizador** da comunidade.

In [ ]:
# Importando as bibliotecas do Tokenizador
from transformers import AutoTokenizer

# Carregando o Tokenizador da comunidade
print('Carregando o tokenizador ' + nome_modelo + ' da comunidade...')

tokenizer = AutoTokenizer.from_pretrained(nome_modelo)

Carregando o tokenizador meta-llama/Llama-2-7b-chat-hf da comunidade...


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Tamanho do vocabulário

In [ ]:
print(len(tokenizer))

32000


## 2.4 - Carregando o LLM

Carregando o **LLM** da comunidade HuggingFace.

Parametrização do from_pretrained
https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu

Carregamento LLama 2 com 4 bits

In [ ]:
# # Importando as bibliotecas do Modelo
# from transformers import BitsAndBytesConfig, AutoModelForCausalLM
# import torch
# import time

# # Guarda o tempo de início do carregamento do modelo
# tempo_inicio = time.time()

# # Carregando o Modelo da comunidade
# print('Carregando o modelo ' + nome_modelo + ' da comunidade...')

# # BitsAndBytes é um framework com funções customizadas para
# # otimização com precisão 8-bit, multiplicações de matrizes e funções de quantização
# quantization_config = BitsAndBytesConfig(
#    load_in_4bit=True, # Habilita a quantização de 4 bits para comprimir o modelo
#    bnb_4bit_quant_type="nf4", # Define o tipo de dados de quantização nas camadas (`fp4` e `nf4`).
#    bnb_4bit_use_double_quant=True, # Quantização aninhada, onde as constantes de quantização da primeira quantização são quantizadas novamente.
#    bnb_4bit_compute_dtype=torch.bfloat16 # # Os gradientes dos pesos são computados em 16-bit. Define o tipo computacional que pode ser diferente do tempo de entrada. Por exemplo, as entradas podem ser fp32, mas a computação pode ser definida como bf16 para acelerações.
# )

# # Se GPU Disponível
# if GPU_ENABLE:
#   # Carrega o modelo com a otimização BitsAndBytesConfig
#   print ("Carregando o LLM com GPU")
#   model = AutoModelForCausalLM.from_pretrained(nome_modelo,
#                                              #torch_dtype=torch.float16, #default
#                                              trust_remote_code=True, # Carrega de um repositório confiável
#                                              quantization_config=quantization_config,
#                                              device_map="auto"
#                                              )
# else:
#   # Carrega o modelo sem a otimização BitsAndBytesConfig
#   print ("Carregando o LLM sem GPU")
#   model = AutoModelForCausalLM.from_pretrained(nome_modelo,
#                                              #torch_dtype=torch.float16, #default
#                                              trust_remote_code=True, # Carrega de um repositório confiável
#                                              device_map="auto"
#                                              )

# # Coloca o modelo e modo avaliação
# model.eval()

# # Aumentar a velocidade
# # https://huggingface.co/docs/transformers/main/perf_torch_compile
# model = torch.compile(model)

# print("Tempo de carregamento do modelo LLM:  {:} (h:mm:ss)".format(formataTempo(time.time() - tempo_inicio)))

Carregamento LLama 2 com 8 bits

In [ ]:
# Importando as bibliotecas do Modelo
from transformers import BitsAndBytesConfig, AutoModelForCausalLM
import torch
import time

# Guarda o tempo de início do carregamento do modelo
tempo_inicio = time.time()

# Carregando o Modelo da comunidade
print('Carregando o modelo ' + nome_modelo + ' da comunidade...')

# BitsAndBytes é um framework com funções customizadas para
# otimização com precisão 8-bit, multiplicações de matrizes e funções de quantização
quantization_config = BitsAndBytesConfig(
   load_in_8bit=True, # Habilita a quantização de 8 bits
)

# Se GPU Disponível
if GPU_ENABLE:
  # Carrega o modelo com a otimização BitsAndBytesConfig
  print ("Carregando o LLM com GPU")
  model = AutoModelForCausalLM.from_pretrained(nome_modelo,
                                             #torch_dtype=torch.float16, #default
                                             trust_remote_code=True, # Carrega de um repositório confiável
                                             quantization_config=quantization_config,
                                             device_map="auto"
                                             )

else:
  # Carrega o modelo sem a otimização BitsAndBytesConfig
  print ("Carregando o LLM sem GPU")
  model = AutoModelForCausalLM.from_pretrained(nome_modelo,
                                             #torch_dtype=torch.float16, #default
                                             trust_remote_code=True, # Carrega de um repositório confiável
                                             device_map="auto"
                                             )

# Coloca o modelo e modo avaliação
model.eval()

# Aumentar a velocidade
# https://huggingface.co/docs/transformers/main/perf_torch_compile
model = torch.compile(model)

print("Tempo de carregamento do modelo LLM:  {:} (h:mm:ss)".format(formataTempo(time.time() - tempo_inicio)))

Carregando o modelo meta-llama/Llama-2-7b-chat-hf da comunidade...
Carregando o LLM com GPU


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Tempo de carregamento do modelo LLM:  0:02:45 (h:mm:ss)


In [ ]:
#print(model)

OptimizedModule(
  (_orig_mod): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(32000, 4096)
      (layers): ModuleList(
        (0-31): 32 x LlamaDecoderLayer(
          (self_attn): LlamaSdpaAttention(
            (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
            (k_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
            (v_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
            (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
            (rotary_emb): LlamaRotaryEmbedding()
          )
          (mlp): LlamaMLP(
            (gate_proj): Linear8bitLt(in_features=4096, out_features=11008, bias=False)
            (up_proj): Linear8bitLt(in_features=4096, out_features=11008, bias=False)
            (down_proj): Linear8bitLt(in_features=11008, out_features=4096, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): LlamaRMSNorm(

In [ ]:
#print(model.config)

LlamaConfig {
  "_name_or_path": "meta-llama/Llama-2-7b-chat-hf",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": false,
    "_load_in_8bit": true,
    "bnb_4bit_compute_dtype": "float32",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "fp4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": false,
    "load_in_8bit": true,
    "quant_method": "bitsandbytes"
  },
  "rms_norm_eps": 1e-05,
  "rope_

In [ ]:
print(model.config.max_position_embeddings)

4096


Tamanho do vocabulário

In [ ]:
print(model.config.vocab_size)

32000


## 2.5 - Configuração da geração de texto

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig

# Instância as configurações do modelo
generation_config = GenerationConfig.from_pretrained(nome_modelo)

print("GenerationConfig antes:\n",generation_config)
generation_config.max_new_tokens = 512 # Preenche até um comprimento máximo especificado com o argumento max_length ou até o comprimento de entrada máximo aceitável para o modelo se esse argumento não for fornecido.
#generation_config.max_length = 4096 # (Default 4096)
# Se do_sample é true setar temperature e top_p, caso contrário se do_sample é false remover temperature e top_p.
generation_config.do_sample = True # (Default True) Se definido como True, este parâmetro permite estratégias de decodificação como amostragem multinomial, amostragem multinomial de busca de feixe, amostragem Top-K e amostragem Top-p. Todas essas estratégias selecionam o próximo token da distribuição de probabilidade em todo o vocabulário com vários ajustes específicos da estratégia.
generation_config.temperature = 0.1 # (Default 0.6) A temperatura é um parâmetro que controla a aleatoriedade da saída do LLM. Uma temperatura mais alta resultará em um texto mais criativo e imaginativo, enquanto uma temperatura mais baixa resultará em um texto mais preciso e factual.
#generation_config.top_k = 3  # Top-k diz ao modelo para escolher o próximo token entre os 'k' tokens principais de sua lista, classificados por probabilidade.
#generation_config.top_p = 0.9 # (Default 0.9) Top-p é mais dinâmico que top-k e é frequentemente usado para excluir resultados com probabilidades mais baixas. Portanto, se você definir p como 0,75, excluirá os 25% inferiores dos resultados prováveis.
#generation_config.repetition_penalty = 1.20 # Penaliza a repetição e visa evitar frases que se repetem sem nada de realmente interessante.
#generation_config.num_return_sequences=1, # Retorna uma única sentença da saída.
generation_config.pad_token_id=generation_config.eos_token_id
print("GenerationConfig depois:\n",generation_config)

GenerationConfig antes:
 GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_length": 4096,
  "pad_token_id": 0,
  "temperature": 0.6,
  "top_p": 0.9
}

GenerationConfig depois:
 GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_length": 4096,
  "max_new_tokens": 512,
  "pad_token_id": 2,
  "temperature": 0.1,
  "top_p": 0.9
}



## 2.6 - Cria o pipeline usando Langchain

Cria o pipeline com a classe [HuggingFacePipeline](https://api.python.langchain.com/en/latest/llms/langchain.llms.huggingface_pipeline.HuggingFacePipeline.html) do langchain.

Passagem direta do pipeline Huggingface.

Configura o pipeline do Huggingface usando o modelo e tokenizador previamente carregado e passa para o HuggingFacePipeline do langchain.

In [ ]:
# Import das bibliotecas
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# Configura o pipeline do HuggingFace
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # return_full_text=True,  # (Default True) Langchain espera o texto completo
    generation_config=generation_config, # Passa as configurações da geração de texto para o pipeline
)

# Carrega o pipeline do Langchain
# https://python.langchain.com/docs/integrations/llms/huggingface_pipelines
model_llm = HuggingFacePipeline(pipeline=pipe)

The model 'OptimizedModule' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MusicgenMelodyForCaus

In [ ]:
print(model_llm)

HuggingFacePipeline
Params: {'model_id': 'gpt2', 'model_kwargs': None, 'pipeline_kwargs': None}


# 3 - Análisando os períodos

Exemplos de tipos de prompts com langchain

* **zero-shot (0-shot) prompts - Solicitação direta**

    Usado quando você deseja que o modelo gere uma resposta sem exemplos. Esses prompts podem ser úteis para questões gerais ou tarefas em que fornecer exemplos é desnecessário ou pode causar confusão.

    Use prompts de disparo 0 quando confiar no conhecimento geral do modelo para fornecer uma resposta suficiente.


* **one-shot (1-shot) prompts - Solicitação com um exemplo**

    Forneça um único exemplo do resultado desejado, ajudando a orientar a resposta do modelo. Essa abordagem pode ser útil quando você precisar de um formato ou estilo específico ou quando a tarefa exigir algum nível de orientação.

    Use prompts únicos quando quiser empurrar o modelo na direção certa sem sobrecarregá-lo com vários exemplos.

* **few-shot (N-shot) prompts - Solicitação com vários  exemplos**

    Ofereça vários exemplos, permitindo que o modelo aprenda com várias instâncias. Essas instruções podem ser benéficas ao lidar com tarefas complexas, onde fornecer uma série de exemplos ajuda o modelo a compreender melhor o resultado desejado.

    Use prompts multi-shot quando um único exemplo pode não ser suficiente para orientar o modelo ou quando você deseja demonstrar um padrão ou tendência.


Referências:
https://anilktalla.medium.com/prompt-engineering-1-shot-prompting-283a0b2b1467

https://www.ssw.com.au/rules/shot-prompts/

https://github.com/awesome-chatgpt-prompts/awesome-chatgpt-prompts-github

Repositório de pompts: https://github.com/awesome-chatgpt-prompts/awesome-chatgpt-prompts-github



## 4.1 - Zero-shot - Solicitação direta

In [ ]:
# Import das bibliotecas
from langchain.chains import LLMChain
from langchain import PromptTemplate

def avaliarPromptZeroShot(texto):

  # Cria o texto de prompt
  prompt_template = """{texto}"""

  # Cria o prompt
  prompt = PromptTemplate(input_variables=["texto"],
                          template = prompt_template)

  # Mostra o prompt
  #prompt_final = prompt.format(text=texto)
  #print(prompt_final)

  # Instancia o chain
  chain = LLMChain(llm=model_llm, prompt=prompt)

  # Executa o prompt no llm
  resultado = chain.invoke(input={"texto": texto})

  return resultado

Texto a ser analisado.

In [ ]:
texto = "Identifique as orações quanto aos períodos compostos (oração principal, oração subordinada, oração coordenada) no texto a seguir:.\
 'Muito gelo, meu Deus, crestou-me as galas!'"

resultado = avaliarPromptZeroShot(texto)

print_linhas_menores(resultado.get('text'))

Identifique as orações quanto aos períodos compostos (oração principal, oração subordinada, oração coordenada) no texto 
a seguir:. 'Muito gelo, meu Deus, crestou-me as galas!' (L.1) 'Crestou-me as galas, meu Deus, muito gelo!' (L.2) 'Muito 
gelo crestou-me as galas, meu Deus!' (L.3) 'Crestou-me as galas, meu Deus, muito gelo!' (L.4) Identifique as orações pri
ncipais, subordinadas e coordenadas em cada uma das linhas (L.1 a L.4).

L.1: Oração principal - 'Muito gelo, meu Deus, 
crestou-me as galas!'
Oração subordinada - 'crestou-me as galas, meu Deus!'
Oração coordenada - 'Muito gelo!'

L.2: Oraç
ão principal - 'Crestou-me as galas, meu Deus, muito gelo!'
Oração subordinada -'mucho gelo!'
Oração coordenada - 'crest
ou-me as galas, meu Deus!'

L.3: Oração principal - 'Muito gelo crestou-me as galas, meu Deus!'
Oração subordinada - 'cr
estou-me as galas, meu Deus!'
Oração coordenada -'muito gelo!'

L.4: Oração principal - 'Crestou-me as galas, meu Deus, 
muito gelo crestou-me as galas!'

## 4.2 - One-shot - Solicitação com um exemplo

In [ ]:
# Import das bibliotecas
from langchain.chains import LLMChain
from langchain import PromptTemplate

def avaliarPromptOneShot(texto):

  # Cria o texto de prompt
  prompt_template = """Identifique as orações quanto aos períodos compostos (oração principal, oração subordinada, oração coordenada). Aqui está um exemplo:
EXEMPLO:
'O homem que chegou a cavalo atirou pro alto.'
Períodos:
'O homem atirou pro alto' -> 'oração principal'
'que estava a cavalo' -> 'oração subordinada'

Agora analise o texto a seguir quanto aos períodos:
'{texto}'
### Resposta:"""

  # Cria o prompt
  prompt = PromptTemplate(input_variables=["texto"],
                          template = prompt_template)

  # Mostra o prompt
  #prompt_final = prompt.format(text=texto)
  #print(prompt_final)

  # Instancia o chain
  chain = LLMChain(llm=model_llm, prompt=prompt)

  # Executa o prompt no llm
  resultado = chain.invoke(input={"texto": texto})

  return resultado

Texto a ser analisado.

In [ ]:
texto = "Muito gelo, meu Deus, crestou-me as galas!"

resultado = avaliarPromptOneShot(texto)

print(resultado.get('text'))

Identifique as orações quanto aos períodos compostos (oração principal, oração subordinada, oração coordenada). Aqui está um exemplo:
Exemplo:
'O homem que chegou a cavalo atirou pro alto.'
Períodos:
'O homem atirou pro alto' -> 'oração principal'
'que estava a cavalo' -> 'oração subordinada'

Agora analise o texto a seguir quanto aos períodos:
'Muito gelo, meu Deus, crestou-me as galas!'
### Resposta:
Períodos:
'Muito gelo' -> 'oração principal'
'meu Deus' -> 'oração subordinada'
'crestou-me as galas' -> 'oração coordenada'

Observação: A oração'meu Deus' é uma oração subordinada, pois é uma oração que descreve o estado de coisa que ocorre no mundo, e não é uma oração que expressa um pensamento ou uma ação do falante.


Texto a ser analisado.

In [ ]:
texto = 'Tu, que foste a vestal dos sonhos d’ouro, o anjo-tutelar dos meus anelos, estende sobre mim as asas brancas. Desenrola os anéis dos teus cabelos! Muito gelo, meu Deus, crestou-me as galas! Muito vento do sul varreu-me as flores! Ai de mim – se o relento de teus risos Não molhasse o jardim dos meus amores!'

resultado = avaliarPromptOneShot(texto)

print(resultado.get('text'))

Identifique as orações quanto aos períodos compostos (oração principal, oração subordinada, oração coordenada). Aqui está um exemplo:
Exemplo:
'O homem que chegou a cavalo atirou pro alto.'
Períodos:
'O homem atirou pro alto' -> 'oração principal'
'que estava a cavalo' -> 'oração subordinada'

Agora analise o texto a seguir quanto aos períodos:
'Tu, que foste a vestal dos sonhos d’ouro, o anjo-tutelar dos meus anelos, estende sobre mim as asas brancas. Desenrola os anéis dos teus cabelos! Muito gelo, meu Deus, crestou-me as galas! Muito vento do sul varreu-me as flores! Ai de mim – se o relento de teus risos Não molhasse o jardim dos meus amores!'
### Resposta:
Períodos:
'Tu, que foste a vestal dos sonhos d’ouro' -> 'oração principal'
'o anjo-tutelar dos meus anelos' -> 'oração subordinada'
'estende sobre mim as asas brancas' -> 'oração principal'
'Desenrola os anéis dos teus cabelos!' -> 'oração subordinada'
'Muito gelo, meu Deus, crestou-me as galas!' -> 'oração principal'
'Muito ven

## 4.3 - Few-shot - Solicitação com vários exemplos

In [ ]:
# Import das bibliotecas
from langchain.chains import LLMChain
from langchain import PromptTemplate

def avaliarPromptFewShot(texto):

  # Cria o texto de prompt
  prompt_template = """Identifique as orações quanto aos períodos compostos (oração principal, oração subordinada, oração coordenada). Aqui está 4 exemplos:
EXEMPLOS:
Exemplo 1: 'O homem que chegou a cavalo atirou pro alto.'
'O homem atirou pro alto' -> 'oração principal'
'que estava a cavalo' -> 'oração subordinada'

Exemplo 2: 'O homem atirou pro alto depois que desceu do cavalo.'
'O homem atirou pro alto' -> 'oração principal
'depois que desceu do cavalo' -> 'oração subordinada'

Exemplo 3: 'O homem chegou a cavalo, desceu e atirou pro alto.'
'O homem atirou chegou a cavalo' -> 'oração coordenada'
'desceu' -> 'oração coordenada'
'e atirou pro alto' -> 'oração coordenada'

Exemplo 4: 'O homem atirou, mas não acertou ninguém.'
'o homem atirou' -> 'oração coordenada'
'mas não acertou ninguém' -> 'oração coordenada'

Agora analise o texto a seguir quanto aos períodos:
'{texto}'
### Resposta:"""

  # Cria o prompt
  prompt = PromptTemplate(input_variables=["texto"],
                          template = prompt_template)

  # Mostra o prompt
  #prompt_final = prompt.format(text=texto)
  #print(prompt_final)

  # Instancia o chain
  chain = LLMChain(llm=model_llm, prompt=prompt)

  # Executa o prompt no llm
  resultado = chain.invoke(input={"texto": texto})

  return resultado

Texto a ser analisado.

In [ ]:
texto = 'Muito gelo, meu Deus, crestou-me as galas!'

resultado = avaliarPromptFewShot(texto)

print(resultado.get('text'))

Identifique as orações quanto aos períodos compostos (oração principal, oração subordinada, oração coordenada). Aqui está 4 exemplos:

Exemplo 1: 'O homem que chegou a cavalo atirou pro alto.'
'O homem atirou pro alto' -> 'oração principal'
'que estava a cavalo' -> 'oração subordinada'

Exemplo 2: 'O homem atirou pro alto depois que desceu do cavalo.'
'O homem atirou pro alto' -> 'oração principal
'depois que desceu do cavalo' -> 'oração subordinada'

Exemplo 3: 'O homem chegou a cavalo, desceu e atirou pro alto.'
'O homem atirou chegou a cavalo' -> 'oração coordenada'
'desceu' -> 'oração coordenada'
'e atirou pro alto' -> 'oração coordenada'

Exemplo 4: 'O homem atirou, mas não acertou ninguém.'
'o homem atirou' -> 'oração coordenada'
'mas não acertou ninguém' -> 'oração coordenada'

Agora analise o texto a seguir quanto aos períodos:
'Muito gelo, meu Deus, crestou-me as galas!'
### Resposta:

Existem 3 períodos no texto:

* 'Muito gelo' -> 'oração principal'
*'meu Deus' -> 'oração su

Texto a ser analisado.

In [ ]:
texto = 'Tu, que foste a vestal dos sonhos d’ouro, o anjo-tutelar dos meus anelos, estende sobre mim as asas brancas. Desenrola os anéis dos teus cabelos! Muito gelo, meu Deus, crestou-me as galas! Muito vento do sul varreu-me as flores! Ai de mim – se o relento de teus risos Não molhasse o jardim dos meus amores!'

resultado = avaliarPromptFewShot(texto)

print(resultado.get('text'))

Identifique as orações quanto aos períodos compostos (oração principal, oração subordinada, oração coordenada). Aqui está 4 exemplos:

Exemplo 1: 'O homem que chegou a cavalo atirou pro alto.'
'O homem atirou pro alto' -> 'oração principal'
'que estava a cavalo' -> 'oração subordinada'

Exemplo 2: 'O homem atirou pro alto depois que desceu do cavalo.'
'O homem atirou pro alto' -> 'oração principal
'depois que desceu do cavalo' -> 'oração subordinada'

Exemplo 3: 'O homem chegou a cavalo, desceu e atirou pro alto.'
'O homem atirou chegou a cavalo' -> 'oração coordenada'
'desceu' -> 'oração coordenada'
'e atirou pro alto' -> 'oração coordenada'

Exemplo 4: 'O homem atirou, mas não acertou ninguém.'
'o homem atirou' -> 'oração coordenada'
'mas não acertou ninguém' -> 'oração coordenada'

Agora analise o texto a seguir quanto aos períodos:
'Tu, que foste a vestal dos sonhos d’ouro, o anjo-tutelar dos meus anelos, estende sobre mim as asas brancas. Desenrola os anéis dos teus cabelos! Muito

## 4.4 - Few-shot - Solicitação com vários exemplos e percentual

In [ ]:
# Import das bibliotecas
from langchain.chains import LLMChain
from langchain import PromptTemplate

def avaliarPromptFewShotP(texto):

  # Cria o texto de prompt
  prompt_template = """Identifique as orações quanto aos períodos compostos (oração principal, oração subordinada, oração coordenada). Aqui está 4 exemplos:
EXEMPLOS:
Exemplo 1: 'O homem que chegou a cavalo atirou pro alto.'
'O homem atirou pro alto' -> 'oração principal'
'que estava a cavalo' -> 'oração subordinada'

Exemplo 2: 'O homem atirou pro alto depois que desceu do cavalo.'
'O homem atirou pro alto' -> 'oração principal'
'depois que desceu do cavalo' -> 'oração subordinada'

Exemplo 3: 'O homem chegou a cavalo, desceu e atirou pro alto.'
'O homem atirou chegou a cavalo' -> 'oração coordenada'
'desceu' -> 'oração coordenada'
'e atirou pro alto' -> 'oração coordenada'

Exemplo 4: 'O homem atirou, mas não acertou ninguém.'
'o homem atirou' -> 'oração coordenada'
'mas não acertou ninguém' -> 'oração coordenada'

Agora analise o texto a seguir quanto aos períodos e identifique o percentual de orações principais e subordinadas:
'{texto}'
### Resposta:"""

  # Cria o prompt
  prompt = PromptTemplate(input_variables=["texto"],
                          template = prompt_template)

  # Mostra o prompt
  #prompt_final = prompt.format(text=texto)
  #print(prompt_final)

  # Instancia o chain
  chain = LLMChain(llm=model_llm, prompt=prompt)

  # Executa o prompt no llm
  resultado = chain.invoke(input={"texto": texto})

  return resultado

Texto a ser analisado.

In [ ]:
texto = 'Tu, que foste a vestal dos sonhos d’ouro, o anjo-tutelar dos meus anelos, estende sobre mim as asas brancas. Desenrola os anéis dos teus cabelos! Muito gelo, meu Deus, crestou-me as galas! Muito vento do sul varreu-me as flores! Ai de mim – se o relento de teus risos Não molhasse o jardim dos meus amores!'

resultado = avaliarPromptFewShot(texto)

print(resultado.get('text'))

Identifique as orações quanto aos períodos compostos (oração principal, oração subordinada, oração coordenada). Aqui está 4 exemplos:

Exemplo 1: 'O homem que chegou a cavalo atirou pro alto.'
'O homem atirou pro alto' -> 'oração principal'
'que estava a cavalo' -> 'oração subordinada'

Exemplo 2: 'O homem atirou pro alto depois que desceu do cavalo.'
'O homem atirou pro alto' -> 'oração principal
'depois que desceu do cavalo' -> 'oração subordinada'

Exemplo 3: 'O homem chegou a cavalo, desceu e atirou pro alto.'
'O homem atirou chegou a cavalo' -> 'oração coordenada'
'desceu' -> 'oração coordenada'
'e atirou pro alto' -> 'oração coordenada'

Exemplo 4: 'O homem atirou, mas não acertou ninguém.'
'o homem atirou' -> 'oração coordenada'
'mas não acertou ninguém' -> 'oração coordenada'

Agora analise o texto a seguir quanto aos períodos:
'Tu, que foste a vestal dos sonhos d’ouro, o anjo-tutelar dos meus anelos, estende sobre mim as asas brancas. Desenrola os anéis dos teus cabelos! Muito